# PART A

In [32]:
import pandas as pd

In [31]:
df = pd.read_csv("ML Lab Exam/Batch 2/train.csv")
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [33]:
# DROPPING NaN VALUES
df=df.dropna()

In [34]:
## GETTING INDEPENDENT FEATURES
x= df.drop('label',axis=1)

In [35]:
# DEPENDENT FEATURE OR TARGET FEATURE
y=df['label']

In [36]:
# LOOKING AT THE SHAPE OF OUR IINDEPENDENT FEATURES DATASET
x.shape

(18285, 4)

In [37]:
# LOOKING AT THE SHAPE OF OUR DEPENDENT FEATURE
y.shape

(18285,)

In [39]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [40]:
# vocabularry size
voc_size=5000

In [41]:
# COPYING OUR INDEPENDENT FEATURE DATASET 'x' INTO NEW VARIABLE 'messages'
messages= x.copy()

In [42]:
# RESETTING THE INDEX VALUES
messages.reset_index(inplace=True)

In [43]:
# IMPORTING LIBRARIES FOR NATURAL LANGUAGE PROCESSING
import nltk
import re
from nltk.corpus import stopwords

In [44]:
# DOWNLOADING THE STOPWORDS
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\dbda-
[nltk_data]     lab\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [45]:
# TEXT PREPROCESSING--> STEMMING, REMOVAL OF STOP WORDS, CONVERTING INTO LOWER CASE
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [46]:
# LOOKING AT OUR FINAL CORPUS
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

### One Hot Encoding

In [47]:
from tensorflow.keras.preprocessing.text import one_hot

In [48]:
onehot_repr= [one_hot(words,voc_size)for words in corpus]
print(onehot_repr)

[[4888, 285, 2368, 4320, 4672, 4988, 501, 2916, 4427, 4732], [2317, 4657, 413, 1151, 876, 4107, 356], [2328, 2368, 523, 2898], [3184, 2021, 3470, 687, 279, 1566], [3838, 876, 3808, 81, 59, 4094, 876, 740, 1947, 4541], [2682, 2117, 4412, 3285, 4572, 2358, 3018, 3736, 3171, 3328, 2387, 1357, 4716, 195, 356], [3866, 1747, 4476, 1871, 3309, 855, 4742, 1683, 2102, 3019, 2657], [3750, 3014, 1044, 3538, 4520, 18, 2358, 857, 2102, 3019, 2657], [3487, 3159, 479, 2124, 4851, 775, 1924, 4761, 2358, 2308], [1419, 398, 3500, 931, 1621, 3474, 55, 2064], [2492, 1561, 1584, 84, 464, 1047, 3741, 114, 2761, 48, 3538], [687, 690, 4672, 775, 2358, 4520], [4362, 4513, 2630, 4160, 365, 4969, 4190, 2204, 2037], [1621, 946, 3336, 1283, 4853, 3058, 2479, 2102, 3019, 2657], [276, 946, 3942, 2126, 1346, 2102, 3019, 2657], [3163, 4276, 4610, 4281, 3895, 4493, 738, 711, 2010, 4722], [2123, 1513, 4657], [3443, 1263, 653, 2822, 2358, 741, 3357, 356], [953, 2118, 413, 1579, 306, 3192, 4854, 1461, 4094], [4239, 2249, 

### Embeddeding Representation

In [49]:
# SETTING OUR SENTENCE LENGTH = 20
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2916 4427 4732]
 [   0    0    0 ...  876 4107  356]
 [   0    0    0 ... 2368  523 2898]
 ...
 [   0    0    0 ... 2102 3019 2657]
 [   0    0    0 ... 3349 2403 2540]
 [   0    0    0 ... 2954  379 2939]]


In [50]:
len(embedded_docs)

18285

### Creating LSTM Model

In [51]:
## CREATING OUR LSTM MODEL
embedding_vector_features= 40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length)) # making embedding layer
model.add(LSTM(100))  # one LSTM Layer with 100 neurons
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [52]:
# CONVERTING X AND Y INTO ARRAYS
import numpy as np
x_final= np.array(embedded_docs)
y_final= np.array(y)

In [53]:
# SPLITTING THE DATA INTO TRAINING AND TEST SETS
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_final,y_final,test_size=0.33,random_state=0)

In [54]:
# FITTING NTO THE MODEL
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 6s 20ms/step - loss: 0.3320 - accuracy: 0.8419 - val_loss: 0.2005 - val_accuracy: 0.9162
Epoch 2/10
192/192 [==============================] - 3s 18ms/step - loss: 0.1440 - accuracy: 0.9465 - val_loss: 0.1956 - val_accuracy: 0.9225
Epoch 3/10
192/192 [==============================] - 3s 18ms/step - loss: 0.0975 - accuracy: 0.9655 - val_loss: 0.2138 - val_accuracy: 0.9118
Epoch 4/10
192/192 [==============================] - 3s 17ms/step - loss: 0.0719 - accuracy: 0.9758 - val_loss: 0.2490 - val_accuracy: 0.9160
Epoch 5/10
192/192 [==============================] - 3s 17ms/step - loss: 0.0451 - accuracy: 0.9856 - val_loss: 0.2866 - val_accuracy: 0.9173
Epoch 6/10
192/192 [==============================] - 3s 17ms/step - loss: 0.0262 - accuracy: 0.9926 - val_loss: 0.3589 - val_accuracy: 0.9157
Epoch 7/10
192/192 [==============================] - 3s 17ms/step - loss: 0.0230 - accuracy: 0.9931 - val_loss: 0.4202 - val_accuracy: 0.9104

In [59]:
predict_x=model.predict(x_test)
y_pred=np.argmax(predict_x,axis=1)

189/189 [==============================] - 1s 4ms/step


In [60]:
y_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [61]:
# IMPORTING LIBRARIES TO SEE THE ACCURACY
from sklearn.metrics import confusion_matrix, accuracy_score

# PRINTING CONFUSION MATRIX
cm= confusion_matrix(y_test,y_pred)
print(cm)

[[3389    0]
 [2646    0]]


In [62]:
# ACCURACY SCORE
ac= accuracy_score(y_test,y_pred)
print(ac)

0.5615575807787904


In [63]:
from tensorflow.keras.layers import Dropout

In [64]:
## CREATING MODEL WITH DROPOUT LAYER
embedding_vector_features= 40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length)) # making embedding layer
model.add(Dropout(0.3))
model.add(LSTM(100))  # one LSTM Layer with 100 neurons
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 40)            200000    
                                                                 
 dropout_3 (Dropout)         (None, 20, 40)            0         
                                                                 
 lstm_2 (LSTM)               (None, 100)               56400     
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [65]:
# CONVERTING X AND Y VARIABLES INTO ARRAYS
import numpy as np
x_final= np.array(embedded_docs)
y_final= np.array(y)

In [66]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_final,y_final,test_size=0.33,random_state=0)

In [71]:
# FITTING INTO THE MODEL
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 3s 18ms/step - loss: 0.0179 - accuracy: 0.9933 - val_loss: 0.3520 - val_accuracy: 0.9175
Epoch 2/10
192/192 [==============================] - 3s 18ms/step - loss: 0.0165 - accuracy: 0.9941 - val_loss: 0.4929 - val_accuracy: 0.9148
Epoch 3/10
192/192 [==============================] - 3s 18ms/step - loss: 0.0129 - accuracy: 0.9956 - val_loss: 0.4732 - val_accuracy: 0.9100
Epoch 4/10
192/192 [==============================] - 3s 18ms/step - loss: 0.0145 - accuracy: 0.9951 - val_loss: 0.5270 - val_accuracy: 0.9059
Epoch 5/10
192/192 [==============================] - 3s 18ms/step - loss: 0.0094 - accuracy: 0.9971 - val_loss: 0.5395 - val_accuracy: 0.9157
Epoch 6/10
192/192 [==============================] - 3s 17ms/step - loss: 0.0072 - accuracy: 0.9978 - val_loss: 0.5383 - val_accuracy: 0.9138
Epoch 7/10
192/192 [==============================] - 3s 18ms/step - loss: 0.0082 - accuracy: 0.9975 - val_loss: 0.5823 - val_accuracy: 0.9150

In [72]:
predict_x=model.predict(x_test)
y_pred=np.argmax(predict_x,axis=1)

189/189 [==============================] - 1s 4ms/step


In [73]:
from sklearn.metrics import confusion_matrix, accuracy_score

# PRINTING CONFUSION MATRIX
cmm= confusion_matrix(y_test,y_pred)
print(cmm)

[[3389    0]
 [2646    0]]


In [74]:
# ACCURACY SCORE
ac= accuracy_score(y_test,y_pred)
print(ac)

0.5615575807787904
